In [ ]:
from collections import defaultdict
import warnings
import pickle
import json
import glob
import os

from benchmark_generation import BenchmarkGenerator
from collision_check import FreqCollisionChecker

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
area_dict = {
    "grid-4": (3000, 3000),
    "grid-25": (7800, 7800),
    "grid-64": (12900, 12900),
    "falcon": (6900,6900),
    "hummingbird": (10800, 10800),
    "eagle" : (15300, 15300),
    "Aspen-11" : (9000, 9000),
    "Aspen-M" : (12900, 12900),
    'xtree-53' : (9300, 9300),
    "xtree-17": (3000, 3000),
    "xtree-5": (2000, 2000),
}

supported = [
    'grid-4',
    'grid-25',  
    'grid-64',
    'falcon', 
    'hummingbird', 
    'eagle',
    'Aspen-11', 
    'Aspen-M', 
    'xtree-53',
    'xtree-17',
    'xtree-5',
]

In [ ]:
for t in supported:
    print("_________________________")
    g = BenchmarkGenerator('benchmark_params.json')
    g(topology=t, substrate_area=area_dict[t])

In [ ]:
result = dict()

for t in supported:
    print(f'Topology: {t}')
    topology_setup_dir = f'test/{t}'
    
    for postfix in os.listdir(topology_setup_dir):
        postfix_dir_path = os.path.join(topology_setup_dir, postfix)
        
        if os.path.isdir(postfix_dir_path):
            print(f'  Postfix: {postfix}')
            files = os.listdir(postfix_dir_path)
            # assert len(files) == 3, f'{files}, missing files'
            testcase_name = f"{t}_{postfix}"
            testcase = defaultdict()
            for cur_file in files:
                file_path = os.path.join(postfix_dir_path, cur_file)
                if file_path.endswith('.log'):
                    continue
                print(f'    file: {file_path}')
                if file_path.endswith(f'{testcase_name}_params.pkl'):
                    with open(file_path, 'rb') as file:
                        testcase['params'] = pickle.load(file)
                elif file_path.endswith(f'{testcase_name}_db.pkl'):
                    with open(file_path, 'rb') as file:
                        testcase['db'] = pickle.load(file)
                elif file_path.endswith(f'{testcase_name}.json'):
                    with open(file_path, 'rb') as file:
                        testcase['json_data'] = json.load(file)
                else:
                    warnings.warn(f"{os.path.basename(file_path)} is not valid file", UserWarning)
                    
            assert all(value is not None for value in testcase.values()), f"Missing pkl/json files for {t}/{postfix}"

            testcase["lef"] = f'{testcase["json_data"]["lef_input"][0]}'
            testcase["def"] = f"results/{testcase_name}/{testcase_name}.gp.def"
            new_def = f"results/{testcase_name}/{testcase_name}.post.def"

            for f in ['lef', 'def']:
                print(f'{f} : {testcase[f]}', end="\t")
                if os.path.isfile(testcase[f]):
                    print('')
                else:
                    print(f'[MISSED]')
            
            collision_checker = FreqCollisionChecker(testcase["def"], testcase["params"], testcase["db"])
            min_bounding_rect = collision_checker.get_min_bounding_rect()
            print(f"Total area of all polygons: {collision_checker.get_total_area()}, Area of MBR: {min_bounding_rect.area}")
            print(f"Minimum bounding rectangle (MBR): {list(min_bounding_rect.exterior.coords)}")
            print('collision length: {} collision area: {} collision percentage: {}\n'.format(
                collision_checker.total_collision_length, 
                collision_checker.total_collision_length*300,
                collision_checker.total_collision_length*300/collision_checker.get_total_area()
                ))
            collision_checker.plot_matrix_layout()
            impactq = collision_checker.plot_collisions()
            collision_checker.check_integration()